In [1]:
import numpy as np
import random
from wordPreprocess import WordPreprocess
from gensim.models import word2vec

In [23]:
programs_questions = []
num_lines = 0
with open('program_quesiton.txt', 'r') as inFile:
    for line in inFile:
        programs_questions.append(line.strip('\n'))
        num_lines += 1
        
        if num_lines % 100000 == 0:
            print ("Finished %d of lines" % num_lines)

word2vec_model = word2vec.Word2Vec.load('word2vec_model.bin')

Finished 100000 of lines
Finished 200000 of lines
Finished 300000 of lines
Finished 400000 of lines
Finished 500000 of lines
Finished 600000 of lines
Finished 700000 of lines
Finished 800000 of lines
Finished 900000 of lines
Finished 1000000 of lines
Finished 1100000 of lines
Finished 1200000 of lines
Finished 1300000 of lines
Finished 1400000 of lines
Finished 1500000 of lines
Finished 1600000 of lines
Finished 1700000 of lines
Finished 1800000 of lines
Finished 1900000 of lines
Finished 2000000 of lines
Finished 2100000 of lines
Finished 2200000 of lines
Finished 2300000 of lines
Finished 2400000 of lines


In [29]:
word_preprocess = WordPreprocess()
doc_hash = word_preprocess.hash_transform(programs_questions, num_feature=2**7)
print (doc_hash.shape)

(2400406, 128)


In [31]:
programs = programs_questions[:2396906]
questions = programs_questions[2396906:]

programs_hash = doc_hash[:2396906]
questions_hash = doc_hash[2396906:]

del doc_hash

In [32]:
def generate_training_data():
    Xs, Ys = [], []
    NUM_TRAIN = 50000
    itr = 0
    
    while (itr < NUM_TRAIN):
        pos_or_neg = random.randint(0,1)
        
        if pos_or_neg is 1:
            line_id = random.randint(0, len(programs)-1)
            
            L1 = programs[line_id].split(' ')[:-1]
            L2 = programs[line_id].split(' ')[:-1]
            
            if (not not L1 and not not L2):
                try:
                    Ys.append(word2vec_model.n_similarity(L1, L2))
                    Xs.append([programs_hash[line_id], programs_hash[line_id+1]])
                    itr += 1
                except KeyError:
                    pass
            
        else:
            line1_id = random.randint(0, len(programs)-1)
            line2_id = random.randint(0, len(programs)-1)
            
            L1 = programs[line1_id].split(' ')[:-1]
            L2 = programs[line2_id].split(' ')[:-1]
            
            if (not not L1 and not not L2):
                try:
                    Ys.append(word2vec_model.n_similarity(L1, L2))
                    Xs.append([programs_hash[line1_id], programs_hash[line2_id]])
                    itr += 1
                except KeyError:
                    pass
            
    return Xs, Ys

In [33]:
from sklearn.model_selection import train_test_split

X, y = generate_training_data()

X_ = []

for t in X:
    X_.append([x_+y_ for x_, y_ in zip(t[0], t[1])])


X_ = np.stack(X_, axis=0)
print (X_)

X_train, X_valid, y_train, y_valid = train_test_split(X_, y, test_size=0.3, random_state=1)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [34]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_valid_std = sc.transform(X_valid)

In [35]:
from sklearn.linear_model import LinearRegression

regr = LinearRegression()
regr.fit(X_train_std, y_train)

y_pred = regr.predict(X_valid_std)

In [36]:
from sklearn.metrics import mean_squared_error


print(mean_squared_error(y_valid, y_pred))

0.100175266324


In [53]:
from sklearn.linear_model import Ridge
lr_rg = Ridge(alpha=a)
lr_rg.fit(X_train_std, y_train)
    
y_pred_rg = lr_rg.predict(X_valid_std)
    
print ("MSE: %.4f", mean_squared_error(y_valid, y_pred_rg))

MSE: %.4f 0.100158037967


from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm_rbf = SVC(kernel='rbf', random_state=0, C=10.0, gamma=0.2)
svm_rbf.fit(X_train_std, y_train)

y_pred = svm_rbf.predict(X_valid_std)

print ("Accuracy: %.3f" % (accuracy_score(y_valid, y_pred)))

In [56]:
import sys

q_test_rg = []
for q_id in range(500):
    question = questions_hash[q_id*7]
    answer = 0
    sim_max = sys.float_info.min
    for i in range(1,7):
        ans = questions_hash[q_id*7 + i]
        set_ = [x+y for x, y in zip(question, ans)]
        set_ = np.array(set_)
        result = lr_rg.predict(set_.reshape(1,-1))
        if result > sim_max:
            answer = i-1
            sim_max = result
        
    q_test_rg.append((q_id, answer))

In [58]:
print (q_test_rg)

[(0, 1), (1, 3), (2, 2), (3, 1), (4, 5), (5, 1), (6, 3), (7, 3), (8, 5), (9, 2), (10, 2), (11, 4), (12, 3), (13, 0), (14, 2), (15, 3), (16, 4), (17, 3), (18, 2), (19, 3), (20, 0), (21, 2), (22, 1), (23, 5), (24, 1), (25, 1), (26, 3), (27, 3), (28, 2), (29, 1), (30, 1), (31, 2), (32, 0), (33, 2), (34, 2), (35, 2), (36, 5), (37, 3), (38, 5), (39, 5), (40, 5), (41, 3), (42, 4), (43, 1), (44, 2), (45, 1), (46, 0), (47, 3), (48, 1), (49, 1), (50, 4), (51, 2), (52, 4), (53, 3), (54, 2), (55, 5), (56, 2), (57, 0), (58, 1), (59, 1), (60, 4), (61, 0), (62, 4), (63, 5), (64, 0), (65, 2), (66, 5), (67, 5), (68, 5), (69, 1), (70, 5), (71, 1), (72, 3), (73, 4), (74, 0), (75, 5), (76, 0), (77, 1), (78, 5), (79, 3), (80, 0), (81, 4), (82, 1), (83, 4), (84, 1), (85, 3), (86, 2), (87, 5), (88, 4), (89, 3), (90, 4), (91, 0), (92, 4), (93, 1), (94, 3), (95, 4), (96, 4), (97, 0), (98, 0), (99, 2), (100, 5), (101, 5), (102, 1), (103, 4), (104, 5), (105, 3), (106, 2), (107, 2), (108, 4), (109, 1), (110, 5),

In [61]:
import pandas as pd

df = pd.DataFrame(data=q_test_rg, columns=['Id', 'Answer']).to_csv('Answer.csv', index=False)